<a href="https://colab.research.google.com/github/chacha86/pythonai2/blob/main/%ED%86%B5%EA%B3%84_%EC%A0%95%EB%B3%B4_%EA%B0%80%EC%A0%B8%EC%98%A4%EA%B8%B0_%ED%92%80%EC%9D%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 패키지 불러오기
import requests
import selenium
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
options.add_argument('headless')
service = ChromeService(executable_path='chromedriver.exe')
driver = webdriver.Chrome(service=service, options=options)

# 실습1
# 뉴스의 링크를 넘기면 해당 뉴스의 댓글 통계 정보를 딕셔너리로 반환하는 함수 만들기

def get_stat_gen(url) :
    driver = webdriver.Chrome(service=service, options=options)
    driver.get(url)

    chart_sex = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "u_cbox_chart_sex"))
    )
#     chart_sex = driver.find_element(By.CLASS_NAME, 'u_cbox_chart_sex')
    per_list = chart_sex.find_elements(By.CLASS_NAME, 'u_cbox_chart_per')

    dic_keys = ['남자', '여자']
    sex_dic = {}

    for key, per in zip(dic_keys, per_list) :
        sex_dic[key] = per.text

    return sex_dic

def get_stat_age(url) :

    driver = webdriver.Chrome(service=service, options=options)
    driver.get(url)
    chart_age = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "u_cbox_chart_age"))
    )
    per_list = chart_age.find_elements(By.CLASS_NAME, 'u_cbox_chart_per')

    age_keys = ['10대', '20대', '30대', '40대', '50대', '60대']
    age_dic = {}

    for key, per in zip(age_keys, per_list) :
        age_dic[key] = per.text

    return age_dic

def get_reply_stat(url) :

    news_id = url.split('?')[0].split('/')[-1]

    gen_info = get_stat_gen(url)
    age_info = get_stat_age(url)

    gen_info.update(age_info)
    gen_info['id'] = news_id


    return gen_info

url = 'https://n.news.naver.com/article/025/0003307263?ntype=RANKING'
stat_info = get_reply_stat(url)
print(stat_info)


import news_util as ut


def get_news_by_journal(journal) :
    target_link = journal['link'] + '&date=20230907'

    page = ut.get_page_by_url(target_link)
    ranking_news = ut.get_rankingnews_by_page(page)

    return ranking_news

sbs = ut.get_journal_by_name('SBS')
kbs = ut.get_journal_by_name('KBS')
mbc = ut.get_journal_by_name('MBC')

journal_list = [sbs, kbs, mbc]

for journal in journal_list :
    ranking_news = get_news_by_journal(journal)
    file_path = journal['name'] + '_news.json'
    ut.save_to_json(file_path, ranking_news)


# 언론사 넘기면 해당 언론사의 랭킹뉴스 20개의 통계 정보를 파일로 저장하는 함수
def get_stat_by_journal(journal) :
    ranking_news = get_news_by_journal(journal) # 언론사의 20개 랭킹 뉴스 가져오기

    stat_list = []
    for news in ranking_news: # 각 20개의 뉴스 링크로 들어가서 통계정보 긁어오기
        try :
            stat_info = get_reply_stat(news['link'])
            stat_list.append(stat_info)
        except : # 긁어오다 문제 생기면 그냥 넘어가기
            print("문제가 발생.. 아마도 대상을 찾지 못한 듯. 일단 넘어감")
    ut.save_to_json(f"{journal['name']}_stat.json", stat_list) # 20개 통계정보 다 찾으면 파일로 저장
    print(f"{journal['name']} 스크랩 작업이 완료되었습니다.") # 알림
